In [1]:
import json
import pandas as pd

In [2]:
country = "France"
year = 2017
geography = "department"

In [3]:
df = pd.read_csv(f"data_output/{country}/{year}_first_round.csv.gz", compression="gzip")
df.columns = [x.lower() for x in df.columns]

df_location = pd.read_csv(f"data_output/{country}/{year}_first_round_location.csv.gz", compression="gzip")

df = pd.merge(df, df_location[["polling_id", geography]], on="polling_id")

In [4]:
df_general = df.groupby("candidate").agg({"value": "sum"}).reset_index()
df_general["rate"] = df_general["value"] / df_general["value"].sum()
df_general.to_json(
    f"C:/Users/cnavarreteliz/Projects/polarization-website/public/data/first_round_{country}_{year}_{geography}.json",
    orient="records"
)

In [5]:
dd = df.groupby(["candidate", geography]).agg({"rank": "std", "value": "sum"})
# dd["asd"] = dd["rate"] * dd["value"] ** 2
dd = dd.groupby("candidate").agg({"rank": "std"})
# dd["d"] = dd["value"] / dd["asd"]
dd.sort_values("rank", ascending=True)

,rank
candidate,
Emmanuel MACRON,19.876148
Jean-Luc MÉLENCHON,20.664084
François FILLON,28.130517
Marine LE PEN,30.284400
Benoît HAMON,31.046485
Nicolas DUPONT-AIGNAN,42.519422
François ASSELINEAU,55.371217
Philippe POUTOU,55.875084
Jean LASSALLE,61.088994


In [6]:
dd = df.groupby("candidate").agg({"rank": "std", "rate": "mean"})
dd["asd"] = dd["rank"] * dd["rate"]
dd.sort_values("asd", ascending=False)

,rank,rate,asd
candidate,,,
Marine LE PEN,98.544009,0.178624,17.602338
François FILLON,85.007034,0.204657,17.397304
Emmanuel MACRON,69.435957,0.249889,17.351264
Jean-Luc MÉLENCHON,73.843956,0.218428,16.129569
Benoît HAMON,120.309455,0.073869,8.887099
Nicolas DUPONT-AIGNAN,152.553154,0.036679,5.595473
Philippe POUTOU,209.257367,0.010301,2.155573
François ASSELINEAU,195.187419,0.010502,2.049929
Jean LASSALLE,220.042872,0.009043,1.989772


In [7]:
df_general = df.groupby("candidate").agg({"rate": "std"}).reset_index().rename(columns={"rate": "value"})
df_general.to_json(
    f"C:/Users/cnavarreteliz/Projects/polarization-website/public/data/dv_{country}_{year}_{geography}.json",
    orient="records"
)

In [8]:
df_general = df.groupby("candidate").agg({"value": "sum"}).reset_index()
df_general["rate"] = df_general["value"] / df_general["value"].sum()
df_general.to_json(
    f"C:/Users/cnavarreteliz/Projects/polarization-website/public/data/first_round_{country}_{year}_{geography}.json",
    orient="records"
)

In [9]:
results = df.groupby("candidate").agg({"value": "sum"}).to_dict()["value"]
results

{'Benoît HAMON': 39787333,
 'Emmanuel MACRON': 137489932,
 'François ASSELINEAU': 5353916,
 'François FILLON': 111123356,
 'Jacques CHEMINADE': 921327,
 'Jean LASSALLE': 4490662,
 'Jean-Luc MÉLENCHON': 115008717,
 'Marine LE PEN': 87527862,
 'Nathalie ARTHAUD': 2864096,
 'Nicolas DUPONT-AIGNAN': 18723465,
 'Philippe POUTOU': 5063293}

In [10]:
df_dv = pd.read_csv(f"data_output/{country}/{year}_divisiveness_{geography}.csv.gz", compression="gzip").fillna(0)
df_dv["rank"] = df_dv.groupby([geography])["value"].rank("min", ascending=False).astype(int)

df_dv.to_json(
    f"C:/Users/cnavarreteliz/Projects/polarization-website/public/data/divisiveness_{country}_{year}_{geography}.json",
    orient="records"
)

In [11]:
df_tmp = df.groupby([geography, "candidate"]).agg({"value": "sum"})
df_tmp["rate"] = df_tmp.groupby(level=[0]).apply(lambda x: x / x.sum())
df_tmp["rank"] = df_tmp.groupby([geography])["value"].rank("min", ascending=False).astype(int)
df_tmp = df_tmp.reset_index()
df_tmp

,department,candidate,value,rate,rank
0,Ain,Benoît HAMON,56932,0.057952,5
1,Ain,Emmanuel MACRON,232180,0.236341,1
2,Ain,François ASSELINEAU,12378,0.012600,7
3,Ain,François FILLON,212741,0.216554,3
4,Ain,Jacques CHEMINADE,1720,0.001751,11
...,...,...,...,...,...
1172,Yvelines,Jean-Luc MÉLENCHON,1606635,0.173832,3
1173,Yvelines,Marine LE PEN,1069742,0.115742,4
1174,Yvelines,Nathalie ARTHAUD,34422,0.003724,10
1175,Yvelines,Nicolas DUPONT-AIGNAN,366895,0.039697,6


In [12]:
df_tmp.to_json(
    f"C:/Users/cnavarreteliz/Projects/polarization-website/public/data/{country}_{year}_{geography}.json",
    orient="records"
)

In [13]:
df_transition = pd.read_csv(f"data_transition/{country}_{year}.csv", encoding="latin-1")
df_transition = df_transition.rename(columns={"Unnamed: 0": "candidate"})
df_transition.columns = [x.replace(".", " ") for x in df_transition.columns]
df_transition

,candidate,Emmanuel MACRON,Marine LE PEN
0,Nicolas DUPONT-AIGNAN,0.000000,0.000000
1,Marine LE PEN,0.000000,0.980303
2,Emmanuel MACRON,0.974019,0.000000
3,Benoît HAMON,0.840257,0.000000
4,Nathalie ARTHAUD,0.000000,0.000000
5,Philippe POUTOU,0.000000,0.000000
6,Jacques CHEMINADE,0.000000,0.000000
7,Jean LASSALLE,0.000000,0.000000
8,Jean-Luc MÉLENCHON,0.936653,0.057970
9,François ASSELINEAU,0.000000,0.000000


In [14]:
candidate_a, candidate_b = list(df_transition)[1:]

In [15]:
output = {
    "links": [],
    "nodes": []
}
for i, tmp in df_transition.iterrows():
    output["links"].append({
        "source": tmp["candidate"],
        "target": f"{candidate_a} 2",
        "rate": tmp[candidate_a],
        "value": results[tmp["candidate"]] * tmp[candidate_a]
    })
    output["links"].append({
        "source": tmp["candidate"],
        "target": f"{candidate_b} 2",
        "rate": tmp[candidate_b],
        "value": results[tmp["candidate"]] * tmp[candidate_b]
    })
    output["nodes"].append({
        "id": tmp["candidate"],
        "candidate": tmp["candidate"]
    })
    
output["nodes"].append({
    "id": f"{candidate_a} 2",
    "candidate": candidate_a
})

output["nodes"].append({
    "id": f"{candidate_b} 2",
    "candidate": candidate_b
})
    

In [16]:
 # Serializing json
json_object = json.dumps(output, indent=4)
 
# Writing to sample.json
with open(f"C:/Users/cnavarreteliz/Projects/polarization-website/public/data/transition_{country}_{year}_{geography}.json", "w") as outfile:
    outfile.write(json_object)